# Check GPU

In [ ]:
import torch
print(torch.cuda.device_count()) 
print(torch.cuda.is_available())
print(torch.__version__)
print(torch.cuda.current_device())

1
True
1.13.1
0


# Fine-tune Whisper

## Prepare Data and Model

### Load Model

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

### Load Datasets

In [ ]:
AUDIO_COLUMN_NAME = "audio"
TEXT_COLUMN_NAME = "sentence"

In [ ]:
from datasets import Audio

def normalize_dataset(ds, audio_column_name=None, text_column_name=None):

    if audio_column_name is not None and audio_column_name != AUDIO_COLUMN_NAME:
        ds = ds.rename_column(audio_column_name, AUDIO_COLUMN_NAME)

    if text_column_name is not None and text_column_name != TEXT_COLUMN_NAME:
        ds = ds.rename_column(text_column_name, TEXT_COLUMN_NAME)

    # resample to the same sampling rate
    ds = ds.cast_column("audio", Audio(sampling_rate=16_000))

    # normalise columns to ["audio", "sentence"]
    ds = ds.remove_columns(set(ds.features.keys()) - set([AUDIO_COLUMN_NAME, TEXT_COLUMN_NAME]))
    
    return ds

### Use huggingface API

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset, DatasetDict

datasets = DatasetDict()

In [ ]:
dataset = load_dataset("mozilla-foundation/common_voice_11_0", "fr", split="test", use_auth_token=True)
dataset = normalize_dataset(dataset)

Found cached dataset common_voice_11_0 (/home/user/.cache/huggingface/datasets/mozilla-foundation___common_voice_11_0/fr/11.0.0/2c65b95d99ca879b1b1074ea197b65e0497848fd697fdb0582e0f6b75b6f4da0)


In [ ]:
dataset

Dataset({
    features: ['audio', 'sentence'],
    num_rows: 16089
})

### Split dataset into train and test

In [ ]:
num_train_examples = 200
num_test_examples = 50

datasets["train"] = dataset.select(range(num_train_examples))
datasets["eval"] =  dataset.select(range(num_test_examples))

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 200
    })
    eval: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 50
    })
})

# Data Augmentation

In [ ]:
from audiomentations import (
    AddBackgroundNoise,
    AddGaussianNoise,
    Compose,
    Gain,
    OneOf,
    PitchShift,
    PolarityInversion,
    TimeStretch,
)

### define augmentation

In [ ]:
augmentation = Compose(
    [
        TimeStretch(min_rate=0.9, max_rate=1.1, p=0.2, leave_length_unchanged=False),
    ]
)

In [ ]:
def augment_dataset(batch):
    # load and (possibly) resample audio data to 16kHz
    sample = batch[AUDIO_COLUMN_NAME]

    # apply augmentation
    augmented_waveform = augmentation(sample["array"], sample_rate=sample["sampling_rate"])
    batch[AUDIO_COLUMN_NAME]["array"] = augmented_waveform
    return batch

### augment training data

In [ ]:
preprocessing_num_workers = 4

augmented_raw_training_dataset = raw_datasets["train"].map(
    augment_dataset, num_proc=preprocessing_num_workers, desc="augment train dataset"
)

### Combine raw data and augment data

In [ ]:
datasets["train"] = concatenate_datasets([datasets["train"], augmented_raw_training_dataset])
datasets["train"] = datasets["train"].shuffle(seed=10)

In [ ]:
datasets

## Preprocessing

In [ ]:
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

normalizer = BasicTextNormalizer()

In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="french", task="transcribe")

In [ ]:
do_normalize_text = True


def prepare_dataset(batch):

    # load
    audio = batch[AUDIO_COLUMN_NAME]

    # compute log-Mel input features from input audio array
    batch["input_features"] = processor.feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # compute input length of audio sample in seconds
    batch["input_length"] = len(audio["array"]) / audio["sampling_rate"]

    # process targets
    input_str = normalizer(batch[TEXT_COLUMN_NAME]).strip() if do_normalize_text else batch[TEXT_COLUMN_NAME]
    
    # encode target text to label ids
    batch["labels"] = processor.tokenizer(input_str).input_ids

    return batch

In [ ]:
vectorized_datasets = datasets.map(
    prepare_dataset, remove_columns=next(iter(datasets.values())).column_names, desc="preprocess dataset"
)

preprocess dataset:   0%|          | 0/200 [00:00<?, ?ex/s]

preprocess dataset:   0%|          | 0/50 [00:00<?, ?ex/s]

## Training and Evaluation

### Data Collator

In [ ]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_features": feature["input_features"]} for feature in features]

        # convert to tensors
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        label_features = [{"input_ids": feature["labels"]} for feature in features]
        
        # pad label ids to the max length in the batch
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

### Evaluation Metrics

In [ ]:
import evaluate

metric = evaluate.load("wer")

In [ ]:
# evaluate with the 'normalised' WER
do_normalize_eval = True


def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    if do_normalize_eval:
        pred_str = [normalizer(pred) for pred in pred_str]

        # perhaps already normalised
        label_str = [normalizer(label) for label in label_str]
        
        # filtering step to only evaluate the samples that correspond to non-zero references
        pred_str = [pred_str[i] for i in range(len(pred_str)) if len(label_str[i]) > 0]
        label_str = [label_str[i] for i in range(len(label_str)) if len(label_str[i]) > 0]

    wer = metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}

### Training Configuration

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./outputs/whisper_medium_ft",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=1,
    warmup_steps=30,
    max_steps=10,
    learning_rate=6.25e-2,
    weight_decay=0.01,
    gradient_checkpointing=True,
    fp16=True,
    predict_with_generate=True,
    generation_max_length=25,
    logging_steps=20,
    report_to=["tensorboard"],
    evaluation_strategy="steps",
    eval_steps=20,
    save_strategy="steps",
    save_steps=20,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
)

### Initialize Trainer

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=vectorized_datasets["train"],
    eval_dataset=vectorized_datasets["eval"],
    tokenizer=processor,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

max_steps is given, it will override any value given in num_train_epochs
Using cuda_amp half precision backend


### GPU check again

In [ ]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

print(f"Using GPU is CUDA:{os.environ['CUDA_VISIBLE_DEVICES']}")

for i in range(torch.cuda.device_count()):
    info = torch.cuda.get_device_properties(i)
    print(f"CUDA:{i} {info.name}, {info.total_memory / 1024 ** 2}MB")

device = torch.device("cuda:0")

Using GPU is CUDA:1
CUDA:0 NVIDIA GeForce RTX 4090, 24195.125MB


### Training

In [ ]:
train_result = trainer.train()

The following columns in the training set don't have a corresponding argument in `WhisperForConditionalGeneration.forward` and have been ignored: input_length. If input_length are not expected by `WhisperForConditionalGeneration.forward`,  you can safely ignore this message.
/home/user/miniconda3/envs/asr_new/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 200
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 10
  Number of trainable parameters = 241734912


  0%|          | 0/10 [00:00<?, ?it/s]

`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Training completed. Do not forget to share your 

{'train_runtime': 9.4258, 'train_samples_per_second': 4.244, 'train_steps_per_second': 1.061, 'train_loss': 9.140096282958984, 'epoch': 0.2}


In [ ]:
model.save_pretrained(training_args.output_dir)
processor.save_pretrained(training_args.output_dir)

Configuration saved in ./outputs/whisper_medium_ft/config.json
Configuration saved in ./outputs/whisper_medium_ft/generation_config.json
Model weights saved in ./outputs/whisper_medium_ft/pytorch_model.bin
Feature extractor saved in ./outputs/whisper_medium_ft/preprocessor_config.json
tokenizer config file saved in ./outputs/whisper_medium_ft/tokenizer_config.json
Special tokens file saved in ./outputs/whisper_medium_ft/special_tokens_map.json
added tokens file saved in ./outputs/whisper_medium_ft/added_tokens.json


## Evaluation

In [ ]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small",  language="french", task="transcribe")

loading file vocab.json from cache at /home/user/.cache/huggingface/hub/models--openai--whisper-small/snapshots/f6744499d1eba717bcf4d6be735e3d386ffb60ad/vocab.json
loading file tokenizer.json from cache at /home/user/.cache/huggingface/hub/models--openai--whisper-small/snapshots/f6744499d1eba717bcf4d6be735e3d386ffb60ad/tokenizer.json
loading file merges.txt from cache at /home/user/.cache/huggingface/hub/models--openai--whisper-small/snapshots/f6744499d1eba717bcf4d6be735e3d386ffb60ad/merges.txt
loading file normalizer.json from cache at /home/user/.cache/huggingface/hub/models--openai--whisper-small/snapshots/f6744499d1eba717bcf4d6be735e3d386ffb60ad/normalizer.json
loading file added_tokens.json from cache at /home/user/.cache/huggingface/hub/models--openai--whisper-small/snapshots/f6744499d1eba717bcf4d6be735e3d386ffb60ad/added_tokens.json
loading file special_tokens_map.json from cache at /home/user/.cache/huggingface/hub/models--openai--whisper-small/snapshots/f6744499d1eba717bcf4d6b

In [ ]:
eval_metrics = trainer.evaluate(metric_key_prefix="eval")

eval_metrics

The following columns in the evaluation set don't have a corresponding argument in `WhisperForConditionalGeneration.forward` and have been ignored: input_length. If input_length are not expected by `WhisperForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 50
  Batch size = 2


  0%|          | 0/25 [00:00<?, ?it/s]

{'eval_loss': 15.846515655517578,
 'eval_wer': 1.0,
 'eval_runtime': 11.0278,
 'eval_samples_per_second': 4.534,
 'eval_steps_per_second': 2.267,
 'epoch': 0.2}